In [1]:
# load libraries
%matplotlib inline
import pandas as pd
import numpy as np
import time
import os

In [2]:
#read in the original file from Tony as a Pandas DataFrame
df = pd.read_csv("IDM_courses_original.csv", header=0)
#show the first five lines to verify upload
df.head()

,FIRST_NAME,LAST_NAME,MATRICULATION_YEAR,COURSE,COURSE_NAME,SEMESTER_TAKEN,YEAR,HGN_LEVEL,NUMBER_OF_UNITS
0,Oladipupo,Doherty,2018,1.001,Eng Computation & Data Science,Spring,2019,G,12
1,Xiaoyu (Sharon),Yan,2018,1.001,Eng Computation & Data Science,Spring,2019,G,12
2,Kevin,Yu,2018,1.001,Eng Computation & Data Science,Spring,2019,G,12
3,Hongling,Chen,2017,1.001,Eng Computation & Data Science,Spring,2018,G,12
4,Manuj,Dhariwal,2015,1.001,Eng Computation & Data Science,Spring,2018,G,12


In [3]:
#Initialize the output dataframe with a dummy row
init_data = [['source','target',0]]
output_df = pd.DataFrame(init_data, columns = ['Source','Target','Value'])
#Verify the new dataframe looks as intended
output_df.head()

,Source,Target,Value
0,source,target,0


In [4]:
#Stupid way to track the correct index of the output DF, but I can't get .append to work
k=0

#iterate through every row in the original file
for i in df.index:
    #pull out the student name and course name into temporary variables
    first = df.at[i,'FIRST_NAME']
    last = df.at[i,'LAST_NAME']
    course = df.at[i,'COURSE_NAME']
    
    #ignore IDM core classes
    if (course == 'Integrated Design Lab I') or (course == 'Integrated Design Lab II') or (course == 'EM Grad Thesis'):
        continue
        
    #Look for instances where the same person took a different class
    for j in df.iloc[(i+1):].index:
        new_course = df.loc[j,'COURSE_NAME']
        
        #ignore IDM core classes
        if (new_course == 'Integrated Design Lab I') or (new_course == 'Integrated Design Lab II') or (new_course == 'EM Grad Thesis'):
            continue
            
        #if the name matches, move it into the output DF with a value of zero
        if (df.at[j,'FIRST_NAME'] == first) and (df.at[j,'LAST_NAME'] == last):
            temp_data = [[course, new_course, 0]]
            temp_df = pd.DataFrame(temp_data, columns = ['Source', 'Target', 'Value'])
            output_df.loc[k] = temp_df.loc[0]
            #move to the next row
            k += 1
        
#show the entire output df to see length
output_df

,Source,Target,Value
0,Eng Computation & Data Science,Business& Operations Analytics,0
1,Eng Computation & Data Science,Data Mining,0
2,Eng Computation & Data Science,Social Media Management,0
3,Eng Computation & Data Science,Fin Data Science & Computing I,0
4,Eng Computation & Data Science,Econ of Information,0
...,...,...,...
12050,Spec Subj in Media Technology,Spec Subj in Media Technology,0
12051,Spec Subj in Media Technology,SpecSubjinMediaTech,0
12052,Spec Subj in Media Technology,"Gender, Health, and Society",0
12053,Spec Subj in Media Technology,Spec Subj in Media Technology,0


In [5]:
#initialize a new DataFrame for cleaning
init_data = [['source','target',0]]
clean_df = pd.DataFrame(init_data, columns = ['Source','Target','Value'])
#verify that it looks as intended
clean_df.head()

,Source,Target,Value
0,source,target,0


In [6]:
#More stupid row tracking by index number
k=0

#Iterate through all rows of the output file
for i in output_df.index:
    #Save the course pairings in temporary variables
    first_course = output_df.at[i,'Source']
    second_course = output_df.at[i,'Target']

    #iterate through the clean Dataframe and see if the course pairing already exists
    repeat_flag = False
    for j in clean_df.index:
        
        #If it does already exist, increment its value by 1 but don't do anything else
        if ((clean_df.at[j,'Source'] == first_course) and (clean_df.at[j,'Target'] == second_course)) or ((clean_df.at[j,'Source'] == second_course) and (clean_df.at[j,'Target'] == first_course)):
            clean_df.at[j,'Value'] += 1
            repeat_flag = True
            break
            
    #No need to do anthing else if we found a repeat
    if repeat_flag == True:
        continue
        
    #If we did not find a repeat, add the course pairing to the clean DataFrame
    else:
        temp_data = [[first_course, second_course, 1]]
        temp_df = pd.DataFrame(temp_data, columns = ['Source', 'Target', 'Value'])
        clean_df.loc[k] = temp_df.loc[0]
        #Move down a row
        k += 1

#Show the clean DataFrame to verify length
clean_df

,Source,Target,Value
0,Eng Computation & Data Science,Business& Operations Analytics,2
1,Eng Computation & Data Science,Data Mining,6
2,Eng Computation & Data Science,Social Media Management,2
3,Eng Computation & Data Science,Fin Data Science & Computing I,1
4,Eng Computation & Data Science,Econ of Information,2
...,...,...,...
8320,Principles of Neuroengineering,Spec Subj in Media Technology,2
8321,Spec Subj in Media Technology,Spec Subj in Media Technology,5
8322,Spec Subj in Media Technology,SpecSubjinMediaTech,2
8323,Spec Subj in Media Technology,"Gender, Health, and Society",1


In [8]:
#Save it as a CSV to the local directory
clean_df.to_csv(r'IDM_links2.csv', index = False)